Fact_Checker: 기사 신뢰도 평가 서비스

In [1]:
!pip -q install google-generativeai feedparser trafilatura python-dateutil scikit-learn beautifulsoup4 lxml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 12.4 MB/s eta 0:00:00


In [2]:
import os, re, json, urllib.parse, logging
from typing import List, Dict, Any, Tuple
import feedparser, trafilatura, requests
from bs4 import BeautifulSoup

import google.generativeai as genai
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

logging.getLogger("trafilatura").setLevel(logging.ERROR)

def clean_text(s: str) -> str:
    if not s:
        return ""
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def safe_truncate(s: str, max_chars: int = 8000) -> str:
    if not s:
        return s
    return s[:max_chars]

In [ ]:
GOOGLE_API_KEY = input("Google API Key: ").strip()
if not GOOGLE_API_KEY:
    raise ValueError("API Key required.")
genai.configure(api_key=GOOGLE_API_KEY)

SYSTEM_INSTRUCTION = (
    "너는 한국어 뉴스를 다루는 팩트체커다. "
    "주장(claim)을 명확히 추출하고, 상대 기사와 비교해 사실 일치/불일치, 누락/과장 여부를 근거와 함께 밝힌다. "
    "출력은 가능하면 간결하고 구조화한다."
)
MODEL_NAME = "gemini-2.5-pro"
model = genai.GenerativeModel(MODEL_NAME, system_instruction=SYSTEM_INSTRUCTION)

In [4]:
try:
    from trafilatura.metadata import extract_metadata as t_extract_metadata
except Exception:
    t_extract_metadata = None

def fetch_article(url: str) -> Dict[str, Any]:
    title, text = "", ""
    try:
        downloaded = trafilatura.fetch_url(url, no_ssl=True)
        if downloaded:
            justtext = trafilatura.extract(
                downloaded,
                include_comments=False,
                include_tables=False,
                favor_recall=True
            )
            if justtext:
                text = clean_text(justtext)
            if t_extract_metadata:
                try:
                    meta = t_extract_metadata(downloaded)
                    if meta and getattr(meta, "title", None):
                        title = meta.title
                except Exception:
                    pass
        if not text:
            resp = requests.get(
                url, timeout=12,
                headers={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/120 Safari/537.36"}
            )
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, "lxml")
            root = soup.find("article") or soup.find(attrs={"role":"main"}) or soup
            ps = [p.get_text(" ", strip=True) for p in root.find_all("p")]
            if not ps:
                ps = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
            text = clean_text(" ".join(ps))
            if not title:
                if soup.title and soup.title.string:
                    title = soup.title.get_text(strip=True)
                else:
                    og = soup.find("meta", property="og:title")
                    if og and og.get("content"):
                        title = og["content"]
        return {"url": url, "title": title or "(제목 없음)", "content": text}
    except Exception as e:
        return {"url": url, "title": "(에러)", "content": "", "error": str(e)}

In [5]:
def extract_keywords_with_gemini(text: str) -> List[str]:
    prompt = (
        "다음 본문에서 핵심 키워드/개체명 5~10개를 한국어로 뽑아줘. "
        "너무 일반적인 단어는 제외하고, 고유명사를 우선한다. "
        "출력은 JSON 배열(문자열 리스트)로만 응답해.\n\n본문:\n" + text[:6000]
    )
    resp = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
    try:
        data = json.loads(resp.text or "[]")
    except Exception:
        data = []
    out, seen = [], set()
    for k in data:
        k2 = clean_text(str(k))
        if k2 and k2.lower() not in seen:
            seen.add(k2.lower()); out.append(k2)
    return out[:10]

In [6]:
def google_news_search_rss_query(q: str, lang="ko", country="KR", max_items=20):
    query = urllib.parse.quote(q)
    url = f"https://news.google.com/rss/search?q={query}&hl={lang}&gl={country}&ceid={country}:{lang}"
    feed = feedparser.parse(url)
    items = []
    for entry in feed.entries[:max_items]:
        items.append({
            "title": entry.get("title", ""),
            "link": entry.get("link", ""),
            "published": entry.get("published", ""),
            "source": entry.get("source", {}).get("title", ""),
        })
    return items, {"rss_url": url, "query": q}

In [7]:
def rank_similar_articles(src_text: str, candidate_links: List[str], top_k=3):
    docs = [src_text]
    metas = []
    for link in candidate_links:
        art = fetch_article(link)
        metas.append(art)
        docs.append(art.get("content",""))
    vectorizer = TfidfVectorizer(max_features=6000, ngram_range=(1,2))
    tfidf = vectorizer.fit_transform(docs)
    sims = cosine_similarity(tfidf[0:1], tfidf[1:]).flatten()
    idxs = sims.argsort()[::-1]
    picked = []
    for i in idxs:
        if len(picked) >= top_k:
            break
        picked.append((metas[i], float(sims[i])))
    return picked

In [8]:
EVAL_SCHEMA = {
  "type": "object",
  "properties": {
    "claims": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "text": {"type": "string"},
          "verdict": {"type": "string", "enum": ["일치", "부분일치", "불일치", "검증불가"]},
          "evidence": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "title": {"type": "string"},
                "url": {"type": "string"},
                "note": {"type": "string"}
              },
              "required": ["url"]
            }
          }
        },
        "required": ["text","verdict","evidence"]
      }
    },
    "omissions_or_exaggerations": {"type": "array", "items": {"type": "string"}},
    "reliability_score": {"type": "number"},
    "summary": {"type": "string"}
  },
  "required": ["claims","reliability_score","summary"]
}

def assess_reliability_with_gemini(src_meta: Dict[str,Any], similars: List[Tuple[Dict[str,Any], float]]) -> Dict[str,Any]:
    src_text = safe_truncate(src_meta.get("content",""), 12000)
    comp = []
    for i,(m,sim) in enumerate(similars, start=1):
        comp.append({
            "rank": i,
            "title": m.get("title",""),
            "url": m.get("url",""),
            "similarity": round(sim, 4),
            "content": safe_truncate(m.get("content",""), 6000)
        })
    schema_json = json.dumps(EVAL_SCHEMA, ensure_ascii=False)
    prompt = (
        "아래 (A) 원문 기사와 (B) 유사 기사들을 비교하여 **JSON**으로만 답하라.\n"
        "- 각 주장(claim)에 대해 verdict(일치/부분일치/불일치/검증불가)와 evidence(제목/URL/메모)를 작성하라.\n"
        "- omissions_or_exaggerations 배열에 누락/과장 포인트를 간결히 담아라.\n"
        "- reliability_score(0~100), summary(<=5줄)를 포함하라.\n"
        f"- JSON 스키마: {schema_json}\n\n"
        f"(A) 원문: 제목={src_meta.get('title','')}, URL={src_meta.get('url','')}\n본문(요약용): {src_text}\n\n"
        f"(B) 유사 기사 Top-{len(comp)}: {json.dumps(comp, ensure_ascii=False)}"
    )
    resp = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
    try:
        data = json.loads(resp.text or "{}")
    except Exception:
        data = {"summary": "(파싱 실패)", "raw": resp.text}
    return data

In [9]:
def _verdict_score(v: str) -> float:
    table = {"일치": 1.0, "부분일치": 0.5, "검증불가": 0.0, "불일치": -1.5}
    return table.get(v, 0.0)

def compute_reliability_score_deterministic(
    evaluation: Dict[str, Any],
    similars: List[Tuple[Dict[str, Any], float]],
    claim_weights: List[float] = None,
    omission_penalty_per_item: float = 5.0,
    omission_penalty_cap: float = 20.0,
    similarity_center: float = 0.50,
    similarity_scale: float = 10.0,
) -> Dict[str, Any]:
    claims = evaluation.get("claims", [])
    if not claims:
        return {"score": 40.0, "notes": "no-claims: fallback"}
    n = len(claims)
    weights = claim_weights if (claim_weights and len(claim_weights) == n) else [1.0]*n
    raw = 0.0
    wsum = 0.0
    for i, c in enumerate(claims):
        v = str(c.get("verdict", "")).strip()
        s = _verdict_score(v)
        w = float(weights[i])
        raw += s * w
        wsum += w
    avg_score = raw / max(wsum, 1e-6)
    base = (avg_score + 1.5) / 2.5 * 100.0
    base = max(0.0, min(100.0, base))
    om_cnt = len(evaluation.get("omissions_or_exaggerations", []) or [])
    om_pen = min(omission_penalty_cap, omission_penalty_per_item * om_cnt)
    after_omissions = max(0.0, base - om_pen)
    if similars:
        avg_sim = sum(s for _, s in similars) / len(similars)
    else:
        avg_sim = similarity_center
    sim_delta = (avg_sim - similarity_center) * (2 * similarity_scale)
    final = max(0.0, min(100.0, after_omissions + sim_delta))
    notes = f"avg={avg_score:.2f} base={base:.1f} -om{om_pen:.1f} simΔ={sim_delta:+.1f} → {final:.1f}"
    return {"score": final, "notes": notes}

In [10]:
def fact_check(url: str, lang="ko", country="KR", max_candidates: int = 15) -> Dict[str,Any]:
    src = fetch_article(url)
    if not src.get("content"):
        raise RuntimeError(f"원문 본문 수집 실패: {src.get('error','본문 비어있음')}")
    keywords = extract_keywords_with_gemini(src["content"])
    query_str = src.get("title") or " ".join(keywords)
    items, search_meta = google_news_search_rss_query(query_str, lang=lang, country=country)
    links = [it["link"] for it in items if it.get("link")]
    picked = rank_similar_articles(src["content"], links[:max_candidates], top_k=3)
    eval_json = assess_reliability_with_gemini(src, picked)
    score_patch = compute_reliability_score_deterministic(eval_json, picked)
    eval_json["reliability_score"] = score_patch["score"]
    eval_json["score_notes"] = score_patch["notes"]
    top3_public = [{"title": m.get("title",""), "url": m.get("url","")} for (m,sim) in picked]
    return {
        "input_url": url,
        "source": src,
        "keywords": keywords,
        "search_meta": search_meta,
        "candidates_found": len(links),
        "top3": top3_public,
        "top3_internal": picked,
        "evaluation": eval_json
    }

타켓 기사 URL 주소를 받아 키워드 추출 및 유사 기사 탐색

In [13]:
input_url = input("뉴스 URL: ").strip()
if not input_url:
    raise ValueError("URL required.")
res = fact_check(input_url)
print("\n키워드:", ", ".join(res["keywords"]))
print("\n유사 기사 Top3:")
for i, item in enumerate(res["top3"], start=1):
    print(f"{i}. {item['url']}")

뉴스 URL: https://www.chosun.com/economy/industry-company/2025/10/30/PMJQJI4IHRGZ3IZ4QLJPTGER74/?outputType=amp

키워드: 젠슨 황, 이재용, 정의선, 엔비디아, 삼성전자, 현대차그룹, 깐부치킨, 치맥, 지포스, 이건희

유사 기사 Top3:
1. https://news.google.com/rss/articles/CBMieEFVX3lxTE41VUlXZG03Wmcwai1OQmlqaGNUZmhWbnpaTWRGclAtZGtkMUh3Q0JUSFMwcTE5Z2hfOVhoYkRISERuWUdCbm5Dd1c0QlVSeU1HQzRrZmlxdEg1enlCdUE3MzM5emUweHlmQ0VCQlR2eFFvZE8xRVV1Sg?oc=5
2. https://news.google.com/rss/articles/CBMiU0FVX3lxTFAwa3l5WUJCQVNPenVfLWFlRFFkcTZFSUd4by11SlRjTklGcmd5NW9MV00tZmVOMEpUQW5SeVFnWjEzbWFwRzM2TUFQSGVYLUY2M1Fj?oc=5
3. https://news.google.com/rss/articles/CBMieEFVX3lxTE5hSjhON29FQVI4ZzJ0MEh2TXMySV9qWWlLaFE2Sng0R0xsQlhWRjNpMDFUVGpoQ0tFZzN6R3JNdmNrTFZObHBwbXhjck1LeVdjN2hidmVsNzhxZUk1QkpBSEo1SE51cXpJQWtWcVVFWERkMnNsY0RnZg?oc=5


결과 확인

In [14]:
from IPython.display import Markdown, display

def render_report(res: Dict[str,Any]):
    ev = res.get("evaluation", {})
    lines = []
    lines.append(f"**신뢰성 점수: {ev.get('reliability_score','-')}/100**")
    lines.append("")
    lines.append("**요약**")
    lines.append(ev.get("summary","(요약 없음)"))
    lines.append("")
    lines.append("**주장별 검증**")
    lines.append("")

    claims = ev.get("claims", [])
    if not claims:
        lines.append("주장 데이터 없음\n")

    for idx, c in enumerate(claims, start=1):
        lines.append(f"주장{idx}: {c.get('text','')}")
        lines.append("")
        lines.append(f"판정: {c.get('verdict','')}")
        lines.append("")
        for e in c.get("evidence", []):
            title = e.get("title","")
            url = e.get("url","")
            note = e.get("note","")
            line = f"근거: [{title or url}]({url})"
            if note:
                line += f" — {note}"
                lines.append("")
            lines.append(line)
        lines.append("")

    sn = ev.get("score_notes","-")
    lines.append(f"<small>{sn}</small>")

    display(Markdown("\n".join(lines)))

render_report(res)


**신뢰성 점수: 75.0/100**

**요약**
젠슨 황 엔비디아 CEO, 이재용 삼성전자 회장, 정의선 현대차그룹 회장이 10월 29일 저녁 서울의 한 치킨집에서 회동했다. 이들은 화기애애한 분위기 속에서 AI 분야 협력을 다짐했으며, 이후 지포스 25주년 행사에 함께 참석해 'AI 깐부' 동맹을 과시했다. 원문 기사는 회동 장면과 대화 내용을 상세히 전했으나, 실제 회동 날짜를 29일이 아닌 30일로 잘못 표기하는 오류가 있었다.

**주장별 검증**

주장1: 젠슨 황, 이재용, 정의선이 10월 30일 서울 삼성동 깐부치킨에서 '치맥' 회동을 가졌다.

판정: 부분일치


근거: [황·이재용·정의선, 삼성동서 '치맥 회동'…AI동맹 과시](https://www.joongang.co.kr/article/25293444) — 다수 언론에서 회동 날짜를 10월 29일 저녁으로 보도했다. 원문 기사가 언급한 30일은 기사 발행일로, 실제 회동 날짜와 차이가 있다.

근거: [[영상] 젠슨 황·이재용·정의선 '깜짝 치맥'…'AI 깐부' 맺었다](https://www.yna.co.kr/view/AKR20241029175351003) — 연합뉴스 역시 회동 시점을 29일 저녁으로 특정하여 보도했다.

주장2: 황 CEO는 지포스 25주년 행사에서 '이재명 대통령 초청으로 APEC에 왔다'고 말했다.

판정: 일치


근거: [젠슨 황 “이재명 대통령 초청으로 와”…‘AI 깐부’ 맺은 이재용·정의선과 ‘치맥’](https://www.hani.co.kr/arti/economy/it/1165158.html) — 원문 기사의 인용은 사실이다. 다수 언론이 해당 발언을 보도했으며, 현직 대통령(윤석열)의 이름을 잘못 말한 '실수'로 분석하고 있다.

주장3: 황 CEO는 이재용, 정의선 회장에게 사인한 위스키와 엔비디아의 초소형 AI 수퍼컴퓨터 'DGX 스파크'를 선물했다.

판정: 일치


근거: [젠슨 황이 이재용·정의선에 선물한 'DGX 스파크' 뭐길래…가격이](https://news.mt.co.kr/mtview.php?no=2024103009131920786) — 선물 품목(하쿠슈 25년 위스키, DGX 스파크)과 사인 등의 세부 내용은 다른 언론 보도와 일치한다.

주장4: 이재용 회장 측이 치킨집 현장에 있던 다른 손님들의 저녁 식사 비용 약 180만원을 결제했다.

판정: 일치


근거: [젠슨 황 "1차는 이재용이 쏜다"…180만원 긁었다](https://www.hankyung.com/article/202410295115g) — 황 CEO가 '1차는 이들이 쏜다'고 말한 것과 이재용 회장 측이 약 180만원을 결제했다는 일화는 여러 매체에서 공통적으로 다루고 있다.

<small>avg=0.88 base=95.0 -om10.0 simΔ=-10.0 → 75.0</small>

<small>끝.</small>